In [1]:
# pip install llama-index llama-index-embeddings-huggingface llama-index-llms-huggingface gradio transformers torch

from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from llama_index.core import VectorStoreIndex, get_response_synthesizer, Settings
from llama_index.core.schema import TextNode
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

import torch, gradio as gr

from openai import OpenAI
from dotenv import load_dotenv
import os
from IPython.display import display, Markdown


# 1. Local embeddings
# Settings.embed_model = HuggingFaceEmbedding("sentence-transformers/all-MiniLM-L6-v2")

# 2. Local LLM (tiny model for CPU)
# model_name = "google/flan-t5-small"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# Settings.llm = HuggingFaceLLM(model=model, tokenizer=tokenizer, max_new_tokens=128,context_window=512,generate_kwargs={"temperature": 0.1})

# 3. Load documents
docs = SimpleDirectoryReader("Data/").load_data()



In [2]:
# import sk from .env file

load_dotenv()

# connect to openai API
client = OpenAI(api_key=os.getenv("BOBAD_OPENAI_KEY"))

In [3]:
# 4. Build index
index = VectorStoreIndex.from_documents(docs)

print(f"Embedding Model: {index._embed_model.model_name}")
print(f"Index Size: {len(index.vector_store.data.embedding_dict)}")
first_key = list(index.vector_store.data.embedding_dict.keys())[0]
first_embedding = index.vector_store.data.embedding_dict[first_key]
print(f"First embedding shape: {len(first_embedding)}")



Embedding Model: text-embedding-ada-002
Index Size: 20
First embedding shape: 1536


In [69]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# changing embedding model
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

In [4]:
index = VectorStoreIndex(docs)

print(f"Embedding Model: {index._embed_model.model_name}")
print(f"Index Size: {len(index.vector_store.data.embedding_dict)}")
first_key = list(index.vector_store.data.embedding_dict.keys())[0]
first_embedding = index.vector_store.data.embedding_dict[first_key]
print(f"First embedding shape: {len(first_embedding)}")

Embedding Model: text-embedding-ada-002
Index Size: 20
First embedding shape: 1536


In [5]:
# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)

In [6]:
results = retriever.retrieve("what is Domain Driven Design?")

In [7]:
display(results)

[NodeWithScore(node=Document(id_='b62c15ae-eb15-4507-ad1c-bfc576e273bc', embedding=None, metadata={'page_label': '1', 'file_name': 'A Crash Course on Domain-Driven Design.pdf', 'file_path': '/Users/dubeys/Documents/ABB/CortexLab/Assignment-3/Data/A Crash Course on Domain-Driven Design.pdf', 'file_type': 'application/pdf', 'file_size': 4062016, 'creation_date': '2024-10-08', 'last_modified_date': '2024-10-08'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='A Crash Course on Domain-Driven Design\nAUG 01, 2024\n1 21 Share\nDeveloping so\x00ware for complex domains is a challenging task.\xa0\nAs the complexity of the problem domain grows, it

In [12]:
# format results in markdown
results_markdown = ""
for i, result in enumerate(results, start=1):
    doc = result.node  # Access the Document inside NodeWithScore
    text_snippet = doc.text_resource.text  # Extract text
    metadata = doc.metadata  # Extract metadata dictionary

    # Example: get file_name or page_label safely
    file_name = metadata.get("file_name", "N/A")
    page_label = metadata.get("page_label", "N/A")

    results_markdown += f"{i}. **File:** {file_name}  \n"
    results_markdown += f"   **Page:** {page_label}  \n"
    results_markdown += f"   **Snippet:** {text_snippet[:500]}...  \n\n"  # Limit snippet to first 500 chars


In [13]:
display(Markdown(results_markdown))

1. **File:** A Crash Course on Domain-Driven Design.pdf  
   **Page:** 1  
   **Snippet:** A Crash Course on Domain-Driven Design
AUG 01, 2024
1 21 Share
Developing so ware for complex domains is a challenging task. 
As the complexity of the problem domain grows, it becomes increasingly di cult to create
so ware that accurately represents the business concepts, rules, and processes. Poorly designed
so ware can quickly turn into an incomprehensible tangle of code that is di cult to understand,
maintain, and extend.
Domain-Driven Design (DDD) o ers a solution to this problem.
DDD is an ...  

2. **File:** A Crash Course on Domain-Driven Design.pdf  
   **Page:** 19  
   **Snippet:** DDD focuses on creating a rich domain model that re ects a deep understanding of the
business domain. The domain model is based on input from domain experts and serves as a
conceptual framework for the so ware.
A ubiquitous language is cultivated based on the domain model. This language is used
consistently by developers and domain experts in all communication - in conversations,
documentation, and the code itself. Using the ubiquitous language helps keep the model
and implementation aligned.
Th...  

3. **File:** A Crash Course on Domain-Driven Design.pdf  
   **Page:** 2  
   **Snippet:** Domain-Driven Design (DDD) focuses on creating so ware systems that closely align with the
underlying business domain. 
It aims to bridge the gap between the technical implementation and the business requirements
by placing the domain model at the center of the development process.
Core Principles of Domain-Driven Design...  

4. **File:** A Crash Course on Domain-Driven Design.pdf  
   **Page:** 5  
   **Snippet:** In DDD, the domain model is not just a conceptual tool. It is the very foundation of the so ware
design. The structure and behavior of the so ware mirror the structure and behavior of the
model. 
This approach is known as model-driven design.
In practice, it means that the classes, relationships, and behaviors in the code directly
correspond to the concepts, relationships, and rules in the domain model. The design is not
driven by technical concerns or infrastructure details but by the need to e...  

5. **File:** A Crash Course on Domain-Driven Design.pdf  
   **Page:** 14  
   **Snippet:** Factories ensure that the objects they create are always in a consistent and fully initialized
state.
For example, an “OrderFactory” in an e-commerce system can take all the necessary details
(customer, items, quantities, etc.), create the “Order” and “LineItem” instances, link them
together appropriately, and return the resulting Order aggregate to the client.
When dealing with large models in big projects, some strategic design concepts should be
followed to gain the bene ts of domain-driven d...  

6. **File:** A Crash Course on Domain-Driven Design.pdf  
   **Page:** 3  
   **Snippet:** There are three core principles of DDD:
Creating a rich domain model based on input from domain experts
Using a ubiquitous language based on the domain model
Driving the design of the so ware from the domain model
Let's explore each of these principles in more detail.
The foundation of DDD lies in the creation of a rich domain model that accurately captures the
key concepts, relationships, and business rules of the problem domain. This model is not created
in isolation by the development team bu...  

7. **File:** A Crash Course on Domain-Driven Design.pdf  
   **Page:** 20  
   **Snippet:** It would be even better if you included Domain Events as a part of tactical DDD components or
patterns. Though it's a newer concept it still plays a crucial role in the domain model when raising
business logic events in event-driven systems.
LIKE (5) REPLY SHARE
© 2024ByteByteGo ∙ Privacy ∙ Terms ∙ Collection notice
Substackis the home for great culture...  

8. **File:** A Crash Course on Domain-Driven Design.pdf  
   **Page:** 7  
   **Snippet:** persistence, messaging, logging, and other technical services. 
For example, in an e-commerce system, the Domain Layer would contain core business concepts
like Product, Order, and Customer. The UI Layer would have screens for product display,
shopping cart, and checkout. The Application Layer would handle the  ow of the order
placement process. The Infrastructure Layer would provide functionalities like persistence,
messaging, etc.
In Domain-Driven Design (DDD), an entity is a fundamental conce...  

9. **File:** A Crash Course on Domain-Driven Design.pdf  
   **Page:** 6  
   **Snippet:** Typically, a layered architecture consists of four main layers:
User Interface (Presentation) Layer: This layer is responsible for presenting information to
the user and interpreting user commands. It communicates with the Application Layer to
send user requests and receive responses.
Application Layer: The Application Layer coordinates the overall application activity and
orchestrates the data  ow between the UI and the Domain Layer. It does not contain
business logic but delegates it to the Do...  

10. **File:** A Crash Course on Domain-Driven Design.pdf  
   **Page:** 13  
   **Snippet:** These methods allow the client code to interact with the “CustomerRepository” in a way that
mimics a collection of “Customer” objects. 
The client can add, remove, and query customers without worrying about how the data is stored
in the database. Behind the scenes, the repository may use an ORM tool, SQL query, or any other
persistence mechanism to interact with the underlying storage system.
In domain-driven design, a factory is a pattern that encapsulates the creation of complex entities
or ag...  



In [76]:
# 5. Query engine
# query_engine = index.as_query_engine(response_mode="compact")



In [14]:
# configure response synthesizer
response_synthesizer = get_response_synthesizer()

In [15]:
# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)

In [16]:
response = query_engine.query("what is Domain-Driven Design ?")
print(response)


Domain-Driven Design (DDD) is an approach to software development that focuses on modeling the core domain and business logic to create software systems that closely align with the underlying business domain. It emphasizes the importance of creating a rich domain model based on input from domain experts, using a ubiquitous language for communication, and driving the software design from the domain model rather than technical concerns. DDD aims to manage complexity by isolating the domain layer from infrastructure and user interface concerns, utilizing patterns such as entities, value objects, services, aggregates, repositories, and factories to build maintainable and extensible systems.


In [17]:
print(f"LLM: {Settings.llm.model}")
from llama_index.llms.openai import OpenAI

LLM: gpt-3.5-turbo


In [18]:
# changing the global LLM
Settings.llm = OpenAI("gpt-4o")
print(f"LLM: {Settings.llm.model}")

LLM: gpt-4o


In [19]:
# simpler way to make query engine
query_engine = index.as_query_engine()
response = query_engine.query("what is Domain-Driven Design?")
print(response)

Domain-Driven Design (DDD) is an approach to software development that addresses the complexity of problem domains by focusing on modeling the core domain and business logic. It emphasizes creating a rich domain model that reflects a deep understanding of the business domain, with input from domain experts. DDD uses this model as the foundation for software design, ensuring that the software accurately represents business concepts, rules, and processes. It involves building collaboration between technical and domain experts and utilizes a ubiquitous language to maintain alignment between the model and implementation. DDD also manages complexity by isolating the domain layer from other concerns and dividing large models into Bounded Contexts, using various patterns and techniques to handle relationships and maintain clarity in complex systems.


In [20]:
print(f"LLM: {Settings.llm.model}")

LLM: gpt-4o


In [21]:
# 6. Gradio UI
def rag_chat(user_input: str) -> str:
    return str(query_engine.query(user_input))

iface = gr.Interface(
    fn=rag_chat,
    inputs="text",
    outputs="text",
    title="📚 Offline RAG Chatbot",
    description="Ask questions about your local documents (embeddings + LLM are 100% local)."
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
